# Unsorted Point in Polygon

In [ ]:
import logging
import time
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
from distributed import LocalCluster, Client

In [ ]:
# set up data paths
base_path = Path().cwd().parent.parent
data_dir = base_path.joinpath('data')

In [ ]:
# create local dask cluster
cluster = LocalCluster(#silence_logs=logging.ERROR,
                       dashboard_address=':8790',
                       n_workers=4,
                       threads_per_worker=2,
                       memory_limit='5 GB')
client = Client(cluster)
client

In [ ]:
# load contiguous us data
df = dd.read_parquet(data_dir.joinpath('contiguous_us_w_geohash.parquet'), columns=['latitude', 'longitude'], engine='pyarrow')
display(df.head(2))
len(df)

In [ ]:
# load various size subsets of the zip code data as geodataframes
zips_1 = gpd.read_file(data_dir.joinpath('zip_codes', 'zips_1.geojson')).loc[:, ['geometry']]
zips_10 = gpd.read_file(data_dir.joinpath('zip_codes', 'zips_10.geojson')).loc[:, ['geometry']]
zips_100 = gpd.read_file(data_dir.joinpath('zip_codes', 'zips_100.geojson')).loc[:, ['geometry']]
zips_1000 = gpd.read_file(data_dir.joinpath('zip_codes', 'zips_1000.geojson')).loc[:, ['geometry']]
# zips_10000 = gpd.read_file(data_dir.joinpath('zip_codes', 'zips_10000.geojson')).loc[:, ['geometry']]  # fails for the unsorted case

# Point in Polygon Test

In [ ]:
# filter function
def spatial_join(large_data_df, zip_codes_gdf):
    crs = "epsg:4326"
    large_data_gdf = gpd.GeoDataFrame(large_data_df,
                                      geometry=gpd.points_from_xy(large_data_df.longitude,
                                                                  large_data_df.latitude),
                                      crs=crs)
    return gpd.sjoin(large_data_gdf, zip_codes_gdf, how='inner', op='within')

In [ ]:
total_points = len(df)
num_partitions = df.npartitions
num_polygons = []
time_sec = []
num_result_points = []
num_points = None

num_points = len(df.partitions[:num_partitions])
t00 = time.time()
for zip_gdf in [zips_1, zips_10, zips_100]:#, zips_1000, zips_10000, zips_all]:
    num_polygons.append(len(zip_gdf))
    t0 = time.time()
    rdf = df.partitions[:num_partitions].map_partitions(spatial_join, zip_codes_gdf=zip_gdf).compute()
    time_sec.append(time.time() - t0)
    num_result_points.append(len(rdf))
    print(f'num_polygons[-1]: {num_polygons[-1]}, time_sec[-1]: {time_sec[-1]:.0f} s')

In [ ]:
# save summary info to file
results_df = pd.DataFrame({'num_polygons': num_polygons,
                           'num_points': num_points,
                           'num_result_points': num_result_points,
                           'sort_time_sec': 0,
                           'time_min': np.asarray(time_sec)/60})                          
results_df.to_csv(f'unsorted_results_df_{datetime.now()}.csv')
results_df

In [ ]:
# release the dask workers
cluster.scale(0)